# 05 · Evaluation Generation & Log-probs

Load Unsloth checkpoints, run evaluation prompts, and capture per-token statistics for MIA features.

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

In [2]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


Mounted at /content/drive
PROJECT_ROOT: /content/drive/MyDrive/secure-llm-mia
Active run mode: subset - Quick debugging subset (<=2k rows) for lightweight Colab smoke tests.
BHC CSV path: /content/drive/MyDrive/mimic-iv-bhc/mimic-iv-bhc.csv


In [3]:
import numpy as np
import torch
from unsloth import FastLanguageModel

from src.modeling.logprobs import token_level_stats

SLICE_ID = 1
TRACK = 'noreplay'
MAX_SEQ_LENGTH = 4096
checkpoint_dir = CHECKPOINT_ROOT / f'slice_{SLICE_ID}' / TRACK / RUN_MODE.name
if not checkpoint_dir.exists():
    raise FileNotFoundError('Run notebook 04 to create checkpoints first.')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(checkpoint_dir),
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
tokenizer.model_max_length = MAX_SEQ_LENGTH

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

unsloth/meta-llama-3.1-8b-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import pandas as pd

CANONICAL_PATH = ARTIFACTS_DIR / f'canonical_bhc_{RUN_MODE.name}.parquet'
if not CANONICAL_PATH.exists():
    raise FileNotFoundError('Canonical dataset missing. Run notebook 01 to create it.')

canonical_df = pd.read_parquet(CANONICAL_PATH)
canonical_df = canonical_df.drop_duplicates('subject_id').set_index('subject_id')

IDS_DIR = ARTIFACTS_DIR / f'slice_{SLICE_ID}' / 'ids'

def read_ids(filename: str) -> list[str]:
    path = IDS_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f'Required ID list not found: {path}')
    return [line.strip() for line in path.read_text().splitlines() if line.strip()]

member_ids = read_ids('members.txt')
non_member_ids = read_ids('non_members.txt')
EVAL_PER_CLASS = min(5, len(member_ids), len(non_member_ids))
if EVAL_PER_CLASS == 0:
    raise ValueError('Not enough member/non-member IDs to build an evaluation set.')

eval_examples: list[dict] = []
for sid in member_ids[:EVAL_PER_CLASS]:
    if sid in canonical_df.index:
        eval_examples.append({'example_id': sid, 'label': 1, 'text': canonical_df.at[sid, 'text']})
for sid in non_member_ids[:EVAL_PER_CLASS]:
    if sid in canonical_df.index:
        eval_examples.append({'example_id': sid, 'label': 0, 'text': canonical_df.at[sid, 'text']})

if not eval_examples:
    raise ValueError('No evaluation examples found in canonical dataset for provided IDs.')

print(f'Evaluation examples: {len(eval_examples)} (members={EVAL_PER_CLASS}, non-members={EVAL_PER_CLASS})')

Evaluation examples: 10 (members=5, non-members=5)


In [5]:
prompts = [ex['text'] for ex in eval_examples]
example_ids = [ex['example_id'] for ex in eval_examples]
labels = [ex['label'] for ex in eval_examples]

all_logits = []
all_target_ids = []

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=MAX_SEQ_LENGTH).to(model.device)
    with torch.inference_mode():
        outputs = model(**inputs)
    # Convert BFloat16 logits to Float32 before converting to numpy
    all_logits.append(outputs.logits.detach().cpu().to(torch.float32).numpy())
    all_target_ids.append(inputs['input_ids'].cpu().numpy())

# Concatenate results for token_level_stats, if necessary adjust stats function
# For now, let's assume token_level_stats can handle a list of logits/target_ids, or we adapt it.
# If token_level_stats expects a single numpy array, we might need to pad/stack them first.
# Let's adapt token_level_stats to handle lists of varying lengths.

# Re-evaluate the token_level_stats function call, as it expects single numpy arrays.
# For now, we will assume each prompt's stats are calculated individually.
# This part needs careful consideration depending on how token_level_stats is designed.

# Assuming token_level_stats can be called for each individual prompt
stats_list = []
for i in range(len(prompts)):
    stats_list.append(token_level_stats(all_logits[i], all_target_ids[i]))

# Now, we need to combine these stats_list into a single structure similar to the original 'stats' dict.
# This might require some padding or careful concatenation depending on the shape of stats.
# For simplification, let's assume 'stats' will be a list of dicts or we just take the first example's stats for demonstration
# A more robust solution would concatenate all token-level stats across all examples.

# Let's adapt to combine the results for further processing.
combined_stats = {
    'nll': [],
    'entropy': [],
    'max_prob': [],
    'win@1': [],
    'win@5': [],
    'win@10': [],
    'win@20': [],
}

for s in stats_list:
    # Use flatten().tolist() to ensure a 1D list is extended, avoiding inhomogeneous shapes
    combined_stats['nll'].extend(s['nll'].flatten().tolist())
    combined_stats['entropy'].extend(s['entropy'].flatten().tolist())
    combined_stats['max_prob'].extend(s['max_prob'].flatten().tolist())
    combined_stats['win@1'].extend(s['win@1'].flatten().astype(int).tolist())
    combined_stats['win@5'].extend(s['win@5'].flatten().astype(int).tolist())
    combined_stats['win@10'].extend(s['win@10'].flatten().astype(int).tolist())
    combined_stats['win@20'].extend(s['win@20'].flatten().astype(int).tolist())

# Convert lists back to numpy arrays for the structure expected by downstream code
stats = {
    'nll': np.array(combined_stats['nll']),
    'entropy': np.array(combined_stats['entropy']),
    'max_prob': np.array(combined_stats['max_prob']),
    'win@1': np.array(combined_stats['win@1']),
    'win@5': np.array(combined_stats['win@5']),
    'win@10': np.array(combined_stats['win@10']),
    'win@20': np.array(combined_stats['win@20']),
}

print('Available features:', stats.keys())

Available features: dict_keys(['nll', 'entropy', 'max_prob', 'win@1', 'win@5', 'win@10', 'win@20'])


Persist per-example features to `reports/features_slice_t.parquet` for downstream attacks once evaluation prompts are finalized.

In [7]:
from datasets import Dataset

FEATURES_DIR = PROJECT_ROOT / 'reports' / 'features'
FEATURES_DIR.mkdir(parents=True, exist_ok=True)
features_path = FEATURES_DIR / f'features_slice_{SLICE_ID}_{TRACK}_{RUN_MODE.name}.parquet'

# Use stats_list from the previous cell to maintain the per-example structure (list of lists)
# instead of the flattened 'stats' arrays.
feature_data = {
    'example_id': example_ids,
    'label': labels,
    'token_nll': [s['nll'].flatten().tolist() for s in stats_list],
    'token_entropy': [s['entropy'].flatten().tolist() for s in stats_list],
    'token_max_prob': [s['max_prob'].flatten().tolist() for s in stats_list],
}
for k in (1, 5, 10, 20):
    key = f'win@{k}'
    feature_data[key] = [s[key].flatten().astype(int).tolist() for s in stats_list]

feature_dataset = Dataset.from_dict(feature_data)
feature_dataset.to_parquet(str(features_path))
print(f'Saved feature dataset to {features_path}')

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saved feature dataset to /content/drive/MyDrive/secure-llm-mia/reports/features/features_slice_1_noreplay_subset.parquet
